In [ ]:
from mpl_toolkits.mplot3d import Axes3D
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt # plotting
import numpy as np # linear algebra
import os # accessing directory structure
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


In [ ]:
print(os.listdir('../input'))

In [ ]:
# Plot the PCA with either 2 or 3 reduced components
def plotPCA(df, nComponents):
    df = df.select_dtypes(include =[np.number]) # keep only numerical columns
    df = df.dropna('columns') # drop columns with NaN
    if df.shape[1] < nComponents:
        print(f'No PCA graph shown: The number of numeric columns ({df.shape[1]}) is less than the number of PCA components ({nComponents})')
        return
    df = df.astype('float64') # Cast to float for sklearn functions
    df = StandardScaler().fit_transform(df) # Standardize features by removing the mean and scaling to unit variance
    pca = PCA(n_components = nComponents)
    principalComponents = pca.fit_transform(df)
    principalDf = pd.DataFrame(data = principalComponents, columns = ['Principal Component ' + str(i) for i in range(1, nComponents + 1)])
    fig = plt.figure(figsize = (8, 8))
    if (nComponents == 3):
        ax = fig.add_subplot(111, projection = '3d')
        ax.set_xlabel('Principal Component 1', fontsize = 15)
        ax.set_ylabel('Principal Component 2', fontsize = 15)
        ax.set_zlabel('Principal Component 3', fontsize = 15)
        ax.set_title('3 component PCA', fontsize = 20)
        ax.scatter(xs = principalDf.iloc[:, 0], ys = principalDf.iloc[:, 1], zs = principalDf.iloc[:, 2])
    else:
        ax = fig.add_subplot(111)
        ax.set_xlabel('Principal Component 1', fontsize = 15)
        ax.set_ylabel('Principal Component 2', fontsize = 15)
        ax.set_title('2 component PCA', fontsize = 20)
        ax.scatter(x = principalDf.iloc[:, 0], y = principalDf.iloc[:, 1])


In [ ]:
# Histogram of column data
def plotHistogram(df, nHistogramShown, nHistogramPerRow):
    nunique = df.nunique()
    df = df[[col for col in df if nunique[col] > 1 and nunique[col] < 50]] # For displaying purposes, pick columns that have between 1 and 50 unique values
    nRow, nCol = df.shape
    columnNames = list(df)
    nHistRow = (nCol + nHistogramPerRow - 1) / nHistogramPerRow
    plt.figure(num=None, figsize=(6*nHistogramPerRow, 8*nHistRow), dpi=80, facecolor='w', edgecolor='k')
    for i in range(min(nCol, nHistogramShown)):
        plt.subplot(nHistRow, nHistogramPerRow, i+1)
        df.iloc[:,i].hist()
        plt.ylabel('counts')
        plt.xticks(rotation=90)
        plt.title(f'{columnNames[i]} (column {i})')
    plt.tight_layout(pad=1.0, w_pad=1.0, h_pad=1.0)
    plt.show()


In [ ]:
# Correlation matrix
def plotCorrelationMatrix(df, graphWidth):
    filename = df.dataframeName
    df = df.dropna('columns') # drop columns with NaN
    df = df[[col for col in df if df[col].nunique() > 1]] # keep columns where there are more than 1 unique values
    if df.shape[1] < 2:
        print(f'No correlation plots shown: The number of non-NaN or constant columns ({df.shape[1]}) is less than 2')
        return
    corr = df.corr()
    plt.figure(num=None, figsize=(graphWidth, graphWidth), dpi=80, facecolor='w', edgecolor='k')
    corrMat = plt.matshow(corr, fignum = 1)
    plt.xticks(range(len(corr.columns)), corr.columns, rotation=90)
    plt.yticks(range(len(corr.columns)), corr.columns)
    plt.gca().xaxis.tick_bottom()
    plt.colorbar(corrMat)
    plt.title(f'Correlation Matrix for {filename}', fontsize=15)
    plt.show()


In [ ]:
# Scatter and density plots
def plotScatterMatrix(df, plotSize, textSize):
    df = df.select_dtypes(include =[np.number]) # keep only numerical columns
    # Remove rows and columns that would lead to df being singular
    df = df.dropna('columns')
    df = df[[col for col in df if df[col].nunique() > 1]] # keep columns where there are more than 1 unique values
    columnNames = list(df)
    if len(columnNames) > 10: # reduce the number of columns for matrix inversion of kernel density plots
        columnNames = columnNames[:10]
    df = df[columnNames]
    ax = pd.plotting.scatter_matrix(df, alpha=0.75, figsize=[plotSize, plotSize], diagonal='kde')
    corrs = df.corr().values
    for i, j in zip(*plt.np.triu_indices_from(ax, k = 1)):
        ax[i, j].annotate('Corr. coef = %.3f' % corrs[i, j], (0.8, 0.2), xycoords='axes fraction', ha='center', va='center', size=textSize)
    plt.suptitle('Scatter and Density Plot')
    plt.show()


In [ ]:

df1 = pd.read_csv('../input/red-light-camera-locations.csv', delimiter=',')
df2 = pd.read_csv('../input/red-light-camera-violations.csv', delimiter=',')


In [ ]:
groep=pd.DataFrame( df2.groupby(['CAMERA ID','VIOLATION DATE'])[['VIOLATIONS']].sum() ).reset_index()
camera_red=groep.pivot(index='VIOLATION DATE', columns='CAMERA ID', values='VIOLATIONS').fillna(0)
camera_red

## does a camera prevent crossing red light ?
### as politics use to pretend, there is a deterrent effect ?
### the graphs goes up... and down... its not so clear

In [ ]:
camera_red.plot()


## clustering, we see four groups... what is that ?

### its not a day of week effect..
### probably moments where part of the city are occupied by festivities, resulting in groups of camera's that are 'workless'

In [ ]:
plotPCA(camera_red,2)

In [ ]:
from sklearn.cluster import KMeans


In [ ]:
camera_red['dow']=(pd.to_datetime(camera_red.reset_index()['VIOLATION DATE']).dt.dayofweek).values
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=7, n_iter=15, random_state=2)
pca = PCA(n_components =3)
u_=pd.DataFrame(svd.fit_transform(StandardScaler().fit_transform(camera_red.drop('dow',axis=1))))
kmeans = KMeans(n_clusters=4, random_state=0).fit(u_)
kmeans.labels_
u_.plot.scatter(x=0,y=1,c=kmeans.labels_,colormap='viridis')
u_.plot.scatter(x=0,y=1,c=camera_red.dow,colormap='viridis')

In [ ]:
camera_red['label']=kmeans.labels_
camera_red.groupby('label').mean()